In [ ]:
!pip install transformers
!pip install nltk
!pip install bert_score


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 36.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 94.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:

import json
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
from bert_score import score as bert_score

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import json

def load_prompts_targets(filepath):
    prompts = []
    targets = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            entry = json.loads(line)
            prompts.append(entry["prompt"].strip())
            targets.append(entry["diagnosis"].strip())
    return prompts, targets

prompts_zeroshot, targets_zeroshot = load_prompts_targets("/content/basic_prompts.jsonl")


print(f" Loaded {len(prompts_zeroshot)} zero-shot prompts")


 Loaded 10 zero-shot prompts


In [ ]:
def load_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", offload_folder="./offload", torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32)
    return tokenizer, model

In [ ]:
tokenizer, model = load_model('stanford-crfm/BioMedLM')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
model = model.to(device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/267 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/602k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/276k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.23M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/876 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/10.7G [00:00<?, ?B/s]

cuda


In [ ]:
def generate_response(prompt, tokenizer, model, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = model.generate(**inputs, max_new_tokens=50, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def diagnosis_coverage(prediction, label):
    """
    Computes coverage: the fraction of label words that appear in the prediction.
    """
    label_tokens = label.strip().lower().split()
    pred_tokens = prediction.strip().lower().split()

    if not label_tokens:
        return 0.0

    covered_tokens = sum(1 for token in label_tokens if token in pred_tokens)
    return covered_tokens / len(label_tokens)
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
def compute_bleu(reference, prediction):
    smoothie = SmoothingFunction().method4
    ref_tokens = [reference.lower().split()]
    pred_tokens = prediction.lower().split()
    return sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothie)

from bert_score import score as compute_bertscore


In [ ]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score as compute_bertscore

def generate_response(prompt, tokenizer, model, max_new_tokens=50):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, do_sample=False)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

def compute_bleu(reference, prediction):
    smoothie = SmoothingFunction().method4
    ref_tokens = [reference.lower().split()]
    pred_tokens = prediction.lower().split()
    return sentence_bleu(ref_tokens, pred_tokens, smoothing_function=smoothie)

def diagnosis_coverage(prediction, label):
    """
    Computes:
    - Coverage: fraction of label tokens found in prediction.
    - Matched tokens: list of label tokens recovered in prediction.
    """
    label_tokens = label.strip().lower().split()
    pred_tokens = prediction.strip().lower().split()

    if not label_tokens:
        return 0.0, []

    matched_tokens = [token for token in label_tokens if token in pred_tokens]
    coverage = len(matched_tokens) / len(label_tokens)
    return coverage, matched_tokens



In [ ]:

predictions_list = []

for prompt in prompts_zeroshot:
    pred = generate_response(prompt, tokenizer, model)
    predictions_list.append(pred)


diagnosis_coverage_list = []
bleu_scores = []
bert_precisions = []
bert_recalls = []
bert_f1s = []
recovered_tokens_list = []

for label, pred in zip(targets_zeroshot, predictions_list):
    #Diagnosis Coverage + Recovered tokens
    coverage, matched_tokens = diagnosis_coverage(pred, label)
    diagnosis_coverage_list.append(coverage)
    recovered_tokens_list.append(matched_tokens)

    #BLEU Score
    bleu_scores.append(compute_bleu(label, pred))

    #BERTScore: Precision, Recall, F1
    precision, recall, f1 = compute_bertscore([pred], [label], lang="en")
    bert_precisions.append(precision.mean().item())
    bert_recalls.append(recall.mean().item())
    bert_f1s.append(f1.mean().item())

Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:28895 for open-end generation.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some

In [ ]:
results_df = pd.DataFrame({
    "Diagnosis Coverage": diagnosis_coverage_list,
    "BLEU Score": bleu_scores,
    "BERT Precision": bert_precisions,
    "BERT Recall": bert_recalls,
    "BERTScore (F1)": bert_f1s
})

pd.set_option('display.max_colwidth', None)  # show full text without cutting
print(results_df)

print("\n--- Overall Metrics ---")
print(f"Diagnosis Coverage Accuracy: {sum(diagnosis_coverage_list) / len(diagnosis_coverage_list):.2%}")
print(f"Average BLEU Score: {sum(bleu_scores) / len(bleu_scores):.4f}")
print(f"Average BERT Precision: {sum(bert_precisions) / len(bert_precisions):.4f}")
print(f"Average BERT Recall: {sum(bert_recalls) / len(bert_recalls):.4f}")
print(f"Average BERTScore (F1): {sum(bert_f1s) / len(bert_f1s):.4f}")


   Diagnosis Coverage  BLEU Score  BERT Precision  BERT Recall  BERTScore (F1)
0            0.500000    0.000961        0.737896     0.807841        0.771286
1            0.222222    0.001208        0.750694     0.767867        0.759183
2            0.000000    0.000000        0.716781     0.719436        0.718106
3            0.000000    0.000000        0.733817     0.810652        0.770323
4            0.000000    0.000000        0.737315     0.762821        0.749851
5            0.000000    0.000000        0.729505     0.721126        0.725291
6            0.200000    0.000984        0.757669     0.820788        0.787967
7            0.000000    0.000000        0.725979     0.793478        0.758229
8            0.333333    0.001007        0.732727     0.733821        0.733274
9            0.000000    0.000000        0.728158     0.715430        0.721738

--- Overall Metrics ---
Diagnosis Coverage Accuracy: 12.56%
Average BLEU Score: 0.0004
Average BERT Precision: 0.7351
Average BERT